# Dog-Cat-Rabbit Image Classifier

#### The objective is to create an image classifier that can accurately differentiate between dogs, cats, and rabbits. I picked these animals because I happened to have 10000 images of dogs and cats and my girlfriend has a rabbit, ergo an easy test subject.

#### As can be expected, we're going to be building a Convolutional Neural Network the specs of which are listed below:

### Neural Net Specs

#### Hidden Layer 1:
filters=32, kernel_size=3, activation='relu', input_shape=[64,64,3], padding='same'
pool_size=2, strides=1

#### Hidden Layer 2:
filters=64, kernel_size=3, activation='relu', padding='same'
pool_size=2, strides=1

#### Hidden Layer 3:
filters=128, kernel_size=3, activation='relu', padding='same'
pool_size=2, strides=1
Dropout(0.25)

#### Fully Connected Output Layer:
units=512, activation='relu'
Dropout(0.5)
units=3, activation='softmax'
optimizer = 'SGD', loss = 'categorical_crossentropy', metrics = ['accuracy']

### Machine Specs:
MacBook Pro (Retina, 13-inch, Late 2013) | 2.4 GHz Dual-Core Intel Core i5 | 8 GB 1600 MHz DDR3 | Intel Iris 1536 MB

In [31]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator

In [32]:
tf.__version__

'2.2.0'

# Part 1 - Data Preprocessing

## Preprocessing the Training Set

In [33]:
train_datagen = ImageDataGenerator(
    rescale = 1./255, # Applies feature scaling to our pixels
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
    
    #rotation_range=40,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #fill_mode='nearest'
)

training_set = train_datagen.flow_from_directory(
    '/Dog Cat Rabbit Classifier/dataset/training_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical' # This is binary if you're classifying between two classes
)

Found 12000 images belonging to 3 classes.


## Preprocessing the Test Set

In [34]:
test_datagen = ImageDataGenerator(
    rescale = 1./255, # Applies feature scaling to our pixels
)

test_set = test_datagen.flow_from_directory(
    '/Dog Cat Rabbit Classifier/dataset/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

Found 3000 images belonging to 3 classes.


# Part 2 - Building the CNN

## Initializing the CNN

In [35]:
cnn = tf.keras.models.Sequential()

## Step 1 - Convolution

In [36]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64,64,3], padding='same')) # the 3 refers to us using RGB vs 1 for greyscale
# the Kernel size needs to be odd and determines the dimensions of our filter. If your images are larger than 128x128, then start with kernels of 5 or 7 then work you way down

## Step 2 - Pooling

In [37]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=1))

## Adding a second convolutional layer

In [39]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')) # We only use the input shape when we add our first layer
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=1))

Layers early in the network architecture (i.e., closer to the actual input image) learn fewer convolutional filters while layers deeper in the network (i.e., closer to the output predictions) will learn more filters.
https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/ 

## Adding a third convolutional layer

In [40]:
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same')) # We only use the input shape when we add our first layer
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=1))

from keras.layers import Dropout

cnn.add(Dropout(0.25))

In [41]:
cnn.add(tf.keras.layers.Flatten()) # Turns the output of the previous steps into a 1D vector

## Step 4 - Full Connection

In [42]:
cnn.add(tf.keras.layers.Dense(units=512, activation='relu')) # Units: Number of hidden neurons connecting to this layer, you decide this
cnn.add(Dropout(0.5))

## Step 5 - Output Layer

In [43]:
cnn.add(tf.keras.layers.Dense(units=3, activation='softmax')) # Units = 1 if you're doing binary classification, and activation = sigmoid

# Part 3 - Training the CNN

## Compiling the CNN

In [44]:
cnn.compile(optimizer = 'SGD', loss = 'categorical_crossentropy', metrics = ['accuracy'])

## Training the CNN on the Training set and evaluating it on the Test set

In [46]:
cnn.fit(x = training_set, validation_data = test_set, epochs=50, batch_size=64)

Epoch 1/50
375/375 [==============================] - 1818s 5s/step - loss: 1.0403 - accuracy: 0.4565 - val_loss: 0.9755 - val_accuracy: 0.4953
Epoch 2/50
375/375 [==============================] - 1778s 5s/step - loss: 0.9736 - accuracy: 0.5203 - val_loss: 0.9144 - val_accuracy: 0.5603
Epoch 3/50
375/375 [==============================] - 2259s 6s/step - loss: 0.9257 - accuracy: 0.5545 - val_loss: 0.8605 - val_accuracy: 0.6097
Epoch 4/50
375/375 [==============================] - 1562s 4s/step - loss: 0.8822 - accuracy: 0.5906 - val_loss: 0.8320 - val_accuracy: 0.6210
Epoch 5/50
375/375 [==============================] - 1596s 4s/step - loss: 0.8440 - accuracy: 0.6153 - val_loss: 0.8247 - val_accuracy: 0.6163
Epoch 6/50
375/375 [==============================] - 1352s 4s/step - loss: 0.8074 - accuracy: 0.6373 - val_loss: 0.7984 - val_accuracy: 0.6313
Epoch 7/50
375/375 [==============================] - 1740s 5s/step - loss: 0.7765 - accuracy: 0.6565 - val_loss: 0.7606 - val_accuracy:

# Part 4 - Making a single prediction

In the below predictions
Lola = Dog,
Gia = Dog,
CJ = Dog,
Luke = Dog,
Bun = Rabbit,

In [115]:
import numpy as np
from keras.preprocessing import image

results = []

index = ["Cat", "Dog", "Rabbit"]

filepath = "/Dog Cat Rabbit Classifier/dataset/single_prediction"

files = os.listdir(filepath)
files.remove(".DS_Store")
files.sort()

for i in files:
    test_image = image.load_img(filepath + "/" + i, target_size=(64, 64))
    test_image = image.img_to_array(test_image)
    test_image = test_image/255
    test_image = np.expand_dims(test_image, axis=0)
    
    result = cnn.predict(test_image)
    probability = result[0].max()
    
    if probability > 0.8:
        prediction = index[np.where(result[0] == result[0].max())[0][0]]
    
    else:
        prediction = "Unknown"

    results.append([prediction, probability])

test_predictions = list(zip([i[4:-4] for i in files], results))
test_predictions

[('Cat', ['Unknown', 0.74151987]),
 ('Cat', ['Rabbit', 0.9310766]),
 ('Cat', ['Cat', 0.9337303]),
 ('Dog', ['Dog', 0.99079365]),
 ('Dog', ['Dog', 0.9992816]),
 ('Rabbit', ['Rabbit', 0.99857974]),
 ('Dog', ['Dog', 0.99999654]),
 ('Bun', ['Rabbit', 0.9753357]),
 ('Bun', ['Cat', 0.8601471]),
 ('Cat', ['Unknown', 0.43681675]),
 ('Cat', ['Cat', 0.9655444]),
 ('Cat', ['Cat', 0.9993455]),
 ('Gia', ['Dog', 0.96553046]),
 ('Rabbit', ['Rabbit', 0.9999423]),
 ('Luke', ['Dog', 0.8733952])]

In [116]:
#cnn.save("dog_cat_rabbit_classifier")

INFO:tensorflow:Assets written to: dog_cat_rabbit_classifier/assets
